In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

pd.set_option('notebook_repr_html', True)

/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
league = ['GB1','FR1','L1','IT1','ES1']
league_page = "https://www.transfermarkt.com/jumplist/startseite/wettbewerb/"

In [4]:
def get_club_details(tr_tag):
    club = tr_tag.find_all('a',class_="vereinprofil_tooltip")[1]
    club_link = club['href']
    club_name = club.get_text()
    club_value = tr_tag.find_all('td',class_="rechts show-for-small show-for-pad nowrap")[0].get_text()
    return tuple((club_link,club_name,club_value))

In [5]:
clubs_list = []
for league_id in league:
    page = requests.get(league_page + league_id,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('tbody')[1]
    tr_container = tbody_container.find_all('tr')
    for tr_tag in tr_container :
        clubs_list.append(get_club_details(tr_tag))
print('All the clubs were uploaded')

All the clubs were uploaded


In [6]:
page = requests.get(league_page + 'GB1',headers = headers)
soup = bs(page.content, 'html.parser')

In [7]:
# soup

In [9]:
# tbody_container = soup.find_all('tbody')[1]

In [8]:
# tbody_container

In [9]:
# tr_container = tbody_container.find_all('tr')

In [10]:
# tr_container

In [11]:
for tr_tag in tr_container:
    club = tr_tag.find_all('a',class_="vereinprofil_tooltip")[1]
    club_link = club['href']
    print(club_link)
    # club_name = club.get_text()
    # club_value = tr_tag.find_all('td',class_="rechts show-for-small show-for-pad nowrap")[0].get_text()

/real-madrid/startseite/verein/418/saison_id/2021
/atletico-madrid/startseite/verein/13/saison_id/2021
/fc-barcelona/startseite/verein/131/saison_id/2021
/fc-sevilla/startseite/verein/368/saison_id/2021
/real-sociedad-san-sebastian/startseite/verein/681/saison_id/2021
/fc-villarreal/startseite/verein/1050/saison_id/2021
/fc-valencia/startseite/verein/1049/saison_id/2021
/real-betis-sevilla/startseite/verein/150/saison_id/2021
/athletic-bilbao/startseite/verein/621/saison_id/2021
/fc-getafe/startseite/verein/3709/saison_id/2021
/celta-vigo/startseite/verein/940/saison_id/2021
/espanyol-barcelona/startseite/verein/714/saison_id/2021
/ud-levante/startseite/verein/3368/saison_id/2021
/ca-osasuna/startseite/verein/331/saison_id/2021
/fc-granada/startseite/verein/16795/saison_id/2021
/rcd-mallorca/startseite/verein/237/saison_id/2021
/deportivo-alaves/startseite/verein/1108/saison_id/2021
/rayo-vallecano/startseite/verein/367/saison_id/2021
/fc-cadiz/startseite/verein/2687/saison_id/2021
/fc

In [12]:
# clubs_list

In [13]:
def get_players_club(player):
    player_id = player['id']
    player_link = player['href']
    player_name = player.get_text()
    return tuple((player_id,player_link,player_name,club_name,club_value))

In [14]:
url_site = "https://www.transfermarkt.com"
player_list = []
for club_link,club_name,club_value in clubs_list:
    page = requests.get(url_site + club_link,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('tbody')[1]
    players_details = tbody_container.find_all('a',class_="spielprofil_tooltip")
    for player in players_details[::2] :
        player_list.append(get_players_club(player))
print('All the players were uploaded')

All the players were uploaded


In [17]:
len(player_list)

2703

In [15]:
def get_profil_detail(soup):
    table_container = soup.find_all('table', class_="auflistung")[0]
    td_container = table_container.find_all('td')
    if td_container[1].find('a') == None:
        birth = td_container[0].find('a')['href'].split("/")[-1]# take it in format YYYY-MM-DD for datetime later
        height = td_container[3].get_text().split("m")[0]# I remove the m of meter 
        country = td_container[4].find('img')['title']
        role = td_container[5].get_text().strip()
        foot = td_container[6].get_text()
    else :
        birth = td_container[1].find('a')['href'].split("/")[-1]
        height = td_container[4].get_text().split("m")[0]# I remove the m of meter 
        country = td_container[5].find('img')['title']
        role = td_container[6].get_text().strip()
        foot = td_container[7].get_text()
    tbody_container = soup.find_all('tbody')[0]
    tr_transfer_container = tbody_container.find_all('tr',class_="zeile-transfer")
    transfer_list = []
    for tr_transfer_tag in tr_transfer_container:
        td_transfer_container = tr_transfer_tag.find_all("td")
        tranfer_from = td_transfer_container[5].get_text()
        transfer_to = td_transfer_container[9].get_text()
        transfer_season = td_transfer_container[0].get_text()
        transfer_date = td_transfer_container[1].get_text()
        transfer_list.append(tuple((tranfer_from,transfer_to,transfer_season,transfer_date)))
    return tuple((Id,name,club,club_value,birth,height,country,role,foot,link.split("/")[1],transfer_list))

In [16]:
player_details = []
i=0
for Id,link,name,club,club_value in player_list:
    i=i+1
    if i%500 == 0:
        print("new league upload")
    try:
        page = requests.get(url_site + link,headers = headers)
        soup = bs(page.content, 'html.parser')
        player_details.append(get_profil_detail(soup))
    except Exception as e:
        player_details.append(tuple((Id,name,club,club_value,None,None,None,None,None,link.split("/")[1],[])))
        continue
print("all player details uploaded")

new league upload
new league upload
new league upload
new league upload
new league upload
all player details uploaded


In [17]:
# Saving player_details as pickle file
import pickle

with open('player_details.pickle', 'wb') as handle:
    pickle.dump(player_details, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [18]:
len(player_details)

2706

In [19]:
# Reading player_details as pickle file

with open('player_details.pickle', 'rb') as handle:
    pickle_player_details = pickle.load(handle)

In [20]:
len(pickle_player_details)

2706

In [21]:
def get_injuries_details(soup):
    tbody_container = soup.find_all('tbody')[0]
    tr_container = tbody_container.find_all('tr')
    injuries_list = []
    for tr_tag in tr_container:
        season = tr_tag.find_all('td')[0]
        injury = tr_tag.find_all('td')[1]
        time_out = tr_tag.find_all('td')[4]
        injuries_list.append(tuple((season.get_text(),injury.get_text(),time_out.get_text().split()[0])))
    return injuries_list

In [22]:
player_list = []
for Id,name,club,club_value,birth,height,country,role,foot,name_link,transfer_list in player_details:
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        page = requests.get("https://www.transfermarkt.com/{}/verletzungen/spieler/{}".format(name_link,Id),headers=headers)
        soup = bs(page.content, 'html.parser')
        player_list.append(tuple((Id,name,club,club_value,birth,height,country,role,foot,transfer_list,get_injuries_details(soup))))
    except Exception as e:
        player_list.append(tuple((Id,name,club,club_value,birth,height,country,role,foot,transfer_list,[])))
        continue
print("all player injuries details uploaded")
print("End of uploading from Transfermarkt")

all player injuries details uploaded
End of uploading from Transfermarkt


In [23]:
pd.DataFrame(player_list)

,0,1,2,3,4,5,6,7,8,9,10
0,238223,Ederson,Manchester City,€44.91m,None,None,None,None,None,[],"[(20/21, Virus Infection, 16), (19/20, Ill, 8)..."
1,221624,Zack Steffen,Manchester City,€44.91m,None,None,None,None,None,[],"[(21/22, Corona virus, 16), (20/21, Bone Bruis..."
2,14555,Scott Carson,Manchester City,€44.91m,None,None,None,None,None,[],"[(20/21, Virus Infection, 67), (12/13, Unknown..."
3,258004,Rúben Dias,Manchester City,€44.91m,None,None,None,None,None,[],"[(20/21, Knock, 4), (17/18, Ankle Injury, 11),..."
4,176553,Aymeric Laporte,Manchester City,€44.91m,None,None,None,None,None,[],"[(21/22, Fitness, 8), (20/21, Strain, 21), (20..."
...,...,...,...,...,...,...,...,...,...,...,...
2701,225593,Pere Milla,Elche CF,€1.88m,None,None,None,None,None,[],"[(17/18, Cheekbone Fracture, 30)]"
2702,92693,Darío Benedetto,Elche CF,€1.88m,None,None,None,None,None,[],"[(19/20, Achilles tendon problems, 8), (19/20,..."
2703,334222,Lucas Boyé,Elche CF,€1.88m,None,None,None,None,None,[],"[(20/21, Knock, 14), (19/20, Inguinal Hernia, ..."
2704,73185,Lucas Pérez,Elche CF,€1.88m,None,None,None,None,None,[],"[(18/19, Ankle Injury, 6), (18/19, Foot Injury..."


In [ ]:
def get_list_club_lequipe(h2_tag,league_id):
    if league_id == 'EQ_D1' :
        club = h2_tag.find_all('a')[1]
        club_link = club['href']
        club_name = club['title']
        return tuple((club_link,club_name))
    else:
        club = h2_tag.find('a')
        club_link = club['href']
        club_name = club['title']
        return tuple((club_link,club_name))

In [ ]:
league = ['EQ_ANG','EQ_D1','EQ_ALL','EQ_ITA','EQ_ESP']
league_page = "https://www.lequipe.fr/Football/"
clubs_list_lequipe = []
for league_id in league:
    print(league_id)
    page = requests.get(league_page + league_id + ".html",headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('div',class_="listeclubs")[0]
    h2_container = tbody_container.find_all('h2')
    for h2_tag in h2_container :
        clubs_list_lequipe.append(get_list_club_lequipe(h2_tag,league_id))
print('All the club were uploaded')

EQ_ANG
EQ_D1


IndexError: ignored

In [ ]:
url_site = "https://www.lequipe.fr"
player_list_lequipe = []
for club_link,club_name in clubs_list_lequipe:
    page = requests.get(url_site + club_link,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('table')[4]
    players_details = tbody_container.find_all('tr')
    for player in players_details[1:] :
        player_details = player.find_all('a')[0]
        weight = player.find_all('td')[6]
        player_name = player_details.get_text().strip()
        player_list_lequipe.append(tuple((player_name,club_name,weight.get_text())))
print("all player from lequipe were uploaded")

In [ ]:
club_to_player_map = {}
for player in player_list_lequipe:
    if player[1] in club_to_player_map.keys():
        club_to_player_map[player[1]][player[0]] = player[2]
    else:
        club_to_player_map[player[1]] = {}
        club_to_player_map[player[1]][player[0]] = player[2]

In [ ]:
#Just to verify the name
lequipe_clubs = club_to_player_map.keys()
print(lequipe_clubs)

In [ ]:
import difflib

In [ ]:
df_players = pd.DataFrame(player_list)
df_players.head()

,0,1,2,3,4,5,6,7,8,9,10
0,238223,Ederson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 16), (19/20, Ill, 8)..."
1,221624,Zack Steffen,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Corona virus, 16), (20/21, Bone Bruis..."
2,14555,Scott Carson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 67), (12/13, Unknown..."
3,258004,Rúben Dias,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Knock, 4), (17/18, Ankle Injury, 11),..."
4,176553,Aymeric Laporte,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Fitness, 8), (20/21, Strain, 21), (20..."


In [ ]:
Final_player = []
for player_transfertmarkt in player_list:
    try:
        list_of_players_lequipe = club_to_player_map[difflib.get_close_matches(player_transfertmarkt[2], lequipe_clubs,cutoff=0.1)[0]].keys()
        weight_player = club_to_player_map[difflib.get_close_matches(player_transfertmarkt[2], lequipe_clubs,cutoff=0.1)[0]][difflib.get_close_matches(player_transfertmarkt[1][:15], list_of_players_lequipe,cutoff=0.1)[0]]
        Final_player.append(tuple((player_transfertmarkt[0],player_transfertmarkt[1],player_transfertmarkt[2],player_transfertmarkt[3],player_transfertmarkt[4],weight_player,player_transfertmarkt[5],player_transfertmarkt[6],player_transfertmarkt[7],player_transfertmarkt[8],player_transfertmarkt[9],player_transfertmarkt[10])))
    except Exception as e:
        print(player_transfertmarkt[2])
        Final_player.append(tuple((player_transfertmarkt[0],player_transfertmarkt[1],player_transfertmarkt[2],player_transfertmarkt[3],player_transfertmarkt[4],None,player_transfertmarkt[5],player_transfertmarkt[6],player_transfertmarkt[7],player_transfertmarkt[8],player_transfertmarkt[9],player_transfertmarkt[10])))
print("END")
print(Final_player)

Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea

In [ ]:
df_player = pd.DataFrame(Final_player)
df_player.columns = ["id", "name", "club","club_value","birth","height","weight","country","role","foot","transfers","injuries"]
df_player.head()
df_player.shape

(2691, 12)

In [ ]:
df_player[df_player['name'] == 'Figo']

,id,name,club,club_value,birth,height,weight,country,role,foot,transfers,injuries
